In [74]:
import numpy as np
import pandas as pd
from pandas import DataFrame as dff
import matplotlib as mpl
import matplotlib.pyplot as plt
import folium as fl
from folium import plugins
import json

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)

In [53]:
df=pd.read_csv('covid_uls_0114.csv')

df

,Unnamed: 0,거주지,확진자,추가 확진자
0,0,남구,373,1.0
1,1,중구,165,2.0
2,2,북구,126,1.0
3,3,울주군,99,0.0
4,4,동구,93,1.0
5,5,서울,1,0.0
6,6,충북,1,0.0


In [54]:
df=df.drop(df.columns[[0]], axis='columns')

df

,거주지,확진자,추가 확진자
0,남구,373,1.0
1,중구,165,2.0
2,북구,126,1.0
3,울주군,99,0.0
4,동구,93,1.0
5,서울,1,0.0
6,충북,1,0.0


In [61]:
df.rename(columns = {'거주지' : 'SIG_KOR_NM'}, inplace = True)

df

,SIG_KOR_NM,확진자,추가 확진자
0,남구,373,1.0
1,중구,165,2.0
2,북구,126,1.0
3,울주군,99,0.0
4,동구,93,1.0
5,서울,1,0.0
6,충북,1,0.0


In [35]:
geo=json.load(open('SIG.geojson', encoding='utf-8'))

geo

{'type': 'FeatureCollection',
 'name': 'SIG',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'SIG_CD': '31110',
    'SIG_ENG_NM': 'Jung-gu',
    'SIG_KOR_NM': '중구'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[129.34394346757836, 35.596161116394576],
      [129.3445256227063, 35.596133044094444],
      [129.34572335952006, 35.59604160043537],
      [129.34812680946666, 35.59485694905141],
      [129.34800479161788, 35.594491594771185],
      [129.34855385836363, 35.59313073351255],
      [129.3480343714356, 35.59295459418678],
      [129.34868789724152, 35.59191270274131],
      [129.3485897002874, 35.58876265177293],
      [129.34819702244684, 35.588761150252815],
      [129.34818318628032, 35.585559328030314],
      [129.3484514323723, 35.584475525971214],
      [129.3485342165583, 35.58270645961616],
      [129.34924861633706, 35.58008119750783],
      [129.35155390912976, 35.57662

In [77]:
for idx, do in enumerate(geo['features']):
    gu = do['properties']['SIG_KOR_NM']
    cp = df.loc[(df.SIG_KOR_NM==gu), '확진자'].iloc[0]
    ncp = df.loc[(df.SIG_KOR_NM==gu), '추가 확진자'].iloc[0]
    
    txt1=f'<b><h4>{gu}</h4></b>확진자 : {cp}'
    txt2=f'<b><h4>{gu}</h4></b>추가 확진자 : {ncp}'
    
    geo['features'][idx]['properties']['tooltip1']=txt1
    geo['features'][idx]['properties']['tooltip2']=txt2
    

In [78]:
map = fl.plugins.DualMap(location=[35.541768, 129.183900], zoom_start=10, tiles='OpenStreetMap')

cp_m=fl.Choropleth(geo_data=geo, name='cp', data=df, columns=('SIG_KOR_NM', '확진자'), key_on='feature.properties.SIG_KOR_NM', fill_color='Oranges', fill_opacity=0.7, line_opacity=0.5, legend_name='확진자').add_to(map.m1)

ncp_m=fl.Choropleth(geo_data=geo, name='cp', data=df, columns=('SIG_KOR_NM', '추가 확진자'), key_on='feature.properties.SIG_KOR_NM', fill_color='Blues', fill_opacity=0.7, line_opacity=0.5, legend_name='추가 확진자').add_to(map.m2)

cp_m.geojson.add_child(fl.features.GeoJsonTooltip(['tooltip1'], labels=False))
ncp_m.geojson.add_child(fl.features.GeoJsonTooltip(['tooltip2'], labels=False))
fl.LayerControl().add_to(map.m1)

map